In [9]:
# %load func2.py
"""
Created on Fri Sep 20 14:19:44 2019

@author: gloriabruin
"""
import numpy as np
from netCDF4 import Dataset
import os.path as op
import datetime as dt
import matplotlib.pyplot as plt
#from func import plot_skewT
from metpy.units import units
import metpy.calc as mpcalc
import os
import pandas as pd
from scipy.stats import linregress,sem
import metpy.constants as mpconsts
from metpy.calc import lcl,el
from metpy.plots import  SkewT
from metpy.calc import (dewpoint_from_specific_humidity,
                        dewpoint_from_relative_humidity,
                        mixing_ratio_from_relative_humidity,
                        relative_humidity_from_dewpoint,
                        find_intersections,el,#lfc,
                        parcel_profile,                        
                        lcl,
                        potential_temperature,
                        equivalent_potential_temperature,
                        specific_humidity_from_mixing_ratio,saturation_vapor_pressure,saturation_mixing_ratio,
                        saturation_equivalent_potential_temperature,
                        moist_lapse,
                        dry_lapse,virtual_potential_temperature,
                        virtual_temperature)
from numba import njit
def trans_time(time_series):
    return pd.DatetimeIndex(np.datetime64('1970-01-01')+np.timedelta64(1,'s')*time_series)

def wide_spines(ax,width = 2):
     ax.spines['top'].set_linewidth(width)
     ax.spines['right'].set_linewidth(width)
     ax.spines['bottom'].set_linewidth(width)
     ax.spines['left'].set_linewidth(width)

def get_entrainment_scheme(alt,entrainment_opt = 1,plum_env_level = 0):
    ## 
    """
      input:  
            alt:  altitude from surface (m)
            entrainment_opt: setting of entrainment rate scheme, 0 is no entrain,1 is DIA, 2 is constant scheme
            plum_env_level:  where the entrainment start to heppend , default is 0
      output:
            entrainment rate profile
    """
    X = np.array([0,])
    if entrainment_opt == 1:
        ce       = 0.4 
        X = np.append(X,np.array([ce*(alt[i]-alt[i-1])/(alt[i]-alt[0]) for i in np.arange(1,len(alt))]))
    elif entrainment_opt == 2:
        ce       = 0.0001 ,
        X = np.append(X,np.array([ce*(alt[i]-alt[i-1]) for i in np.arange(1,len(alt))]))
    else:
        x = np.full(shape = (len(alt),),fill_value = 0)
    pick = (alt- alt[0])<plum_env_level
    X[pick] = 0
    return X

def plot_skewT(pressure,temperature,dewpoint, pics = (1,1,1),prof_parcel_temperature = None,fig = None):   
    from metpy.plots import SkewT
    from matplotlib.ticker import MultipleLocator, FormatStrFormatter
    pick0 = (~np.isnan(pressure))
    if np.all(prof_parcel_temperature == None):
        pick = (~np.isnan(pressure))

    else:
        pick = (~np.isnan(pressure)) & (~np.isnan(prof_parcel_temperature))
    #def draw_skewT_buoyancy(p_in,T_in,Td_in,Tp_in=[],bp=[],index=0):
    #print(pressure)  
    #print(pressure[pick0])      
    p  = pressure[pick0] * units('hPa')
    T  = temperature[pick0] * units('degC')
    Td = dewpoint[pick0]* units('degC')
    p1  = pressure[pick] * units('hPa')
    T1  = temperature[pick] * units('degC')
    Td1 = dewpoint[pick]* units('degC')
   
    if fig == None:
    	fig  = plt.figure(figsize=(15,9))
    else:
    	pass
    skew = SkewT(fig,subplot = (pics[0],pics[1],pics[2]),rotation=45)
    
    # Plot the data using normal plotting functions, in this case using
    # log scaling in Y, as dictated by the typical meteorological plot
    skew.plot(p, T, 'r',label='Temperature')
    skew.plot(p, Td, 'g',label='Dew_T')
    # style_list=['-', '--', '-.', ':',]
    # label_list=['NoEntrain','ConstEntrain','DIA'] 
    
    #skew.plot_barbs(p, u, v)
    skew.ax.set_ylim(np.nanmax(p), 100)
    skew.ax.set_xlim(-40, 60)
    #print(p[0], T[0], Td[0])
    # Calculate LCL height and plot as black dot
    lcl_pressure, lcl_temperature = lcl(p[0], T[0], Td[0])
    skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')
    # print('LCL:',lcl_pressure, lcl_temperature)
    # Calculate full parcel profile and add to plot as black line
    prof = parcel_profile(p1, T1[0], Td1[0]).to('degC')
    skew.plot(p1, prof, 'k', linewidth=2,label='Ideal_Pacel_T')
    if not np.any(prof_parcel_temperature) == None:   
        Tp = prof_parcel_temperature[pick]* units('degC')
        skew.plot(p1, Tp,'k',linestyle=':',label='Parcel_T')
        skew.shade_cin(p1, Tp,T1)
        skew.shade_cape(p1,  Tp,T1)
    else:
        skew.shade_cin(p, prof,T)
        skew.shade_cape(p,prof,T)
    #plt.legend()
    
    #Shade areas of CAPE and CIN
    #skew.shade_cin(p, T, prof)
    #skew.shade_cape(p, T, prof)
    
    
    # An example of a slanted line at constant T -- in this case the 0
    # isotherm
    skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)
    
    # Add the relevant special lines
    skew.plot_dry_adiabats()
    skew.plot_moist_adiabats()
    skew.plot_mixing_lines()
    plt.legend()
def blank_skewT():   
    from metpy.plots import SkewT
    from matplotlib.ticker import MultipleLocator, FormatStrFormatter

    fig  = plt.figure(figsize=(15,9))
    skew = SkewT(fig,rotation=30)
    
    #skew.plot_barbs(p, u, v)
    skew.ax.set_ylim(1000, 200)
    skew.ax.set_xlim(-40, 60)
    #print(p[0], T[0], Td[0])
    # Calculate LCL height and plot as black dot
    skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)
    
    # Add the relevant special lines
    # skew.plot_dry_adiabats()
    # skew.plot_moist_adiabats()
    # skew.plot_mixing_lines()
    plt.xlabel('Temperature(degC)',fontsize = 20)
    plt.ylabel('Pressure(hPa)',fontsize = 20)
    plt.legend()
#=============================calculation of buoyance===============================
def _find_append_zero_crossings(x, y):
    r"""
    Find and interpolate zero crossings.

    Estimate the zero crossings of an x,y series and add estimated crossings to series,
    returning a sorted array with no duplicate values.

    Parameters
    ----------
    x : `pint.Quantity`
        x values of data
    y : `pint.Quantity`
        y values of data

    Returns
    -------
    x : `pint.Quantity`
        x values of data
    y : `pint.Quantity`
        y values of data

    """
    # Find and append crossings to the data
    crossings = find_intersections(x[1:], y[1:], np.zeros_like(y[1:]) * y.units)
    x = concatenate((x, crossings[0]))
    y = concatenate((y, crossings[1]))

    # Resort so that data are in order
    sort_idx = np.argsort(x)
    x = x[sort_idx]
    y = y[sort_idx]

    # Remove duplicate data points if there are any
    keep_idx = np.ediff1d(x, to_end=[1]) > 0
    x = x[keep_idx]
    y = y[keep_idx]
    return x, y
def _greater_or_close(a, value, **kwargs):
    r"""Compare values for greater or close to boolean masks.
    """
    return (a > value) | np.isclose(a, value, **kwargs)
def _less_or_close(a, value, **kwargs):
    r"""Compare values for less or close to boolean masks.
    """
    return (a < value) | np.isclose(a, value, **kwargs)
## parameters
class data_struct():
    def __init__(self):
        self.p = []
        self.t = []
        self.h = []
## parameters
g           = 9.81;
p00         = 100000;
cpd         = 1005.7;
Rd          = 287.04;
Rv          = 461.5;
t0          = 273.15;
cpv         = 1875;
cpl         = 4190;
cpi         = 2118.636;
reps        = Rv / Rd;
rddcp       = Rd / cpd;
cpdg        = cpd/g;
converge    = 0.0001;
xlv         = 2501000;
xls         = 2836017;
lv1         = xlv + (cpl - cpv) * t0;
lv2         = cpl - cpv;
ls1         = xls + (cpi - cpv) * t0;
ls2         = cpi - cpv;  
#==============================================================================
@njit
def get_qvs(p,t):
    #t is kelvin degree
    #function to calculate saturated mixing ratio
    #if input is Td instead of T, output is qv    
    es = 611.2 * np.exp(17.67 * (t - 273.15)/(t - 29.65))
    qvs = 287.04/461.5 * es / (p - es) #混合比 大气物理P20,2.2.4
    return qvs

@njit
def get_qvi(p,t):
    #function to calculate saturated water vapor mixing ratio for ice
    #Magnus,1967
    #Bolton 1980, t< 0,大气物理P21-2.2.11, 
    #es  = 610.78 * np.exp(21.8745584 * (t - 276.16) / (t - 7.66))
    es  = 611.2 * np.exp(21.8745584 * (t - 276.16) / (t - 7.66))
    qvi = 287.04/461.5 * es / (p - es)
    return qvi

@njit
def get_entropy(p,t,rv,rl,ri):
    #function to calculate entropy
    #Hauf and Holler 1987 (3.13)
    s0d = 6775; 
    #s0v = 10320; 
    s0l = 3517; 
    #s0i = 2296;
    xlv = 2501000
    xls = 2836017
    lv1 = xlv + (cpl - cpv) * t0
    lv2 = cpl - cpv
    ls1 = xls + (cpi - cpv) * t0
    ls2 = cpi - cpv
    #lv0 = 2501000
    #ls0 = 2836017  
    rt = rv + rl + ri;  # total water mixing ratio
    qd = 1 / (1 + rt);  #dry air percentage (not mixing ratio)
    qt = rt * qd;       #total water specific humidity
    qv = rv * qd;       #vapor specific humidity
    qi = ri * qd;       #ice specific humidity
    # ql = rl * qd;     # liquid specific humidity
    pv = Rv*rv / (Rd + Rv*rv) * p   # partial vapor pressure of parcel
    pd = p - pv                     # partial pressure of dry air
    esl = 611.2  * np.exp(17.67 * (t - t0) / (t - 29.65));
    esi = 610.78 * np.exp(21.8745584 * (t - t0) / (t - 7.66));
    Llv = lv1 - lv2 * t;
    Liv = ls1 - ls2 * t;
    #Llv = lv0 - (cpl - cpv) * (t - t0 )
    #Liv = ls0 - (cpi - cpv) * (t - t0 )
    Lil = Liv - Llv
    Alv = Rv*t * np.log(esl/pv )
    Ail = Rv*t * np.log(esi/esl)
    #print(t/t0)
    #print(pd,p00,pd/p00)
    sd = cpd * np.log(t/t0) - Rd * np.log(pd/p00) + s0d;
    #sv = cpv * log(t/t0) - Rv * log(pv/p00) + s0v;
    sl = cpl * np.log(t/t0) + s0l;
    #si = cpi * log(t/t0) + s0i;
    
    s = sd * qd + sl * qt + ((Llv+Alv)*qv - (Lil+Ail)*qi)/ t;
    #print('S:',s)
    #s = sd * qd + sv * qt + (- (Llv+Alv)*ql - (Liv+Aiv)*qi) / t;
    #s = sd * qd + sv * qv + sl * ql + si * qi;
    return s

#=============================================================================
#=============================================================================
source_list={0:['Start from specified height level'],
             1:['Start from surface'],
             2:['Start from specified pressure level'],}
            
h_input_list={1:['dewpoint'],
             2:['mixing ratio'],
             3:['relative humidity'],}

entrainment_opt_list={0:['No Entrainment'],
                      1:['Constant entrainment rate'],
                      2:['DIA scheme'],}

def get_entrainment_scheme(alt,entrainment_opt = 1,plum_env_level = 0):
    ## 
    """
      input:  
            alt:  altitude from surface (m)
            entrainment_opt: setting of entrainment rate scheme, 0 is no entrain,1 is DIA, 2 is constant scheme
            plum_env_level:  where the entrainment start to heppend , default is 0
      output:
            entrainment rate profile
    """
    X = np.array([0,])
    if entrainment_opt == 1:
        ce = [0.4,] 
        X = np.append(X,np.array([ce[0]*(alt[i]-alt[i-1])/(alt[i]-alt[0]) for i in np.arange(1,len(alt))]))
    elif entrainment_opt == 2:
        ce = [0.0001 ,]
        for i in np.arange(1,len(alt)):
            # print(type(alt[i]-alt[i-1]),type(ce[0]))
            X = np.append(X,ce[0]*(alt[i]-alt[i-1]))

        # X = np.append(X,np.array([ce*float(alt[i]-alt[i-1]) for i in np.arange(1,len(alt))]))
    else:
        X = np.full(shape = alt.shape,fill_value = 0)
    pick = (alt- alt[0])<plum_env_level
    X[pick] = 0
    return X

class cal_buoyancy_simple():                   
    """   
    ===============================================================================
       This model is from Yizhou Zhuang
       Department of Atmospheric and Oceanic Sciences,
       School of Physics, Peking University
       zyz90@pku.edu.cn
       Last updated: 05/11/2017
    ===============================================================================
       Part of this function is ported from a Fortran subroutine written by
       George H. Bryan, which is used in CM-1 model

       References:  Bolton (1980, MWR)             (constants and definitions)
                    Bryan and Fritsch (2004, MWR)  (ice processes)
                    Hauf and Holler (1987, JAS)    (entropy)
                    Holloway and Neelin (2009, JAS)(DIA/DIB entrainment)
    ===============================================================================
   Input:  P_IN   - vector of pressure                                  (hPa) 
           z_IN   - vector of altitude,below ground should be nan       (m) 
           T_IN   - vector of temperature                               (degC/degK) 
           TD_IN    - vector of dewpoint temperature                    (degC/degK)
   Output: 
           BP        - Buoyancy                                     (m/s2)
           TP        - parcel temperature                          (degK)
           QP        - parcel mixing ratio of water vapor, liquid and ice,
                       stored as a n*3 matrix                       (g/kg)
           OTHER     - other variables, stored as a structure 
                       (see below for all available fields), have been delete in this script
               DBPDZS- buoyancy increment during isentropic process
                       1. env lapse rate                            (s^-2)
                       2. dry adiabat
                       3. heat storage by water
                       4. condensation/vaporization
                       5. freezing/melting
                       6. parcel vapor
                       7. env vapor
                       8. liquid
                       9. ice
               DBPDZE- buoyancy increment due to entrainment
                       1. dry air mixing
                       2. heat storage exchange by water
                       3. condensation after mixing
                       4. freezing after mixing   
                       5. vapor during mixing
                       6. liquid during mixing
                       7. ice during mixing
                       8. vapor during phase change
                       9. liquid during phase change
                       10. ice during phase change
               DBPDZA- buoyancy increment due to condensate falled out
                       1. liquid
                       2. ice
               ZC    - calculated height                               (m)
   User option within the code:
           Z         - height of the profile (calculate if not given)  (m)
           SOURCE    - source of parcel
                       0 = specific level (specified by zs)
                       1 = specific mixed layer( default:500 above surface )
                       
           ZS        - parcel origin level (source=0)                  (m)
                       default: surface
           ICE       - ice process option
                       0 = no ice process, liquid only
                       1 = Tao et al 1989
           TICE      - temperature at which condensate all freeze (ice=1)
                       default: -40 degC                            (degC)
           ADIABAT   - formulation of moist adiabat:
                       0 = condensate all kept in parcel
                       1 = condensate all fall out of parcel
                       2 = max condensate specified
                       3 = condensate fall out with a certain ratio
           QW_MAX    - max condensate mixing ratio in cloud (adiabat=2)
                       default: 1                                   (g/kg)
           PRC_W     - percentage of condensate falling out of parcel
                       (adiabat=3) default: 0.5/km                  (km-1)
           ENTRAIN_OPT - entrainment option
                       0 = no entrainment
                       1 = constant entrainment
                       2 = DIA entrainment
           X_EN      - fractional entrainment rate (entrain_opt=1)  (km-1)
                       default: 0.1/km
           CE        - constant related to entrainment (entrain_opt=2)(m-1)
           """
    def __init__(self,
                t_profile,         # temperature profile
                h_profile,         # humidigy profile-units:g/kg
                p_profile,         # units:hPa    
                z_profile,         # alt  unit:m
                X,                 # entrainment rate
                td_r_q   = 1  ,    # input option for humidity profile
                adiabat  = 1,      # Formulation of moist adiabat
                yaxis_base = 0,      # yaxis base, 0: altitude y axis, 1: pressure y axis
                source   = 1,       # decie where to lift, 0 from surface, 1 from mixd layer
                prc_w    = 0.0005 , # percentage per m (0-1) of condensate kept in the parcel
                qc_max   = 0.001,   # max mixing ratio of condensate in cloud
                ice      = 1  ,   # include ice process
                tice     = 233.15 , # temperature at which all condensate will be ice form 
                newton_step_dt   = 10,
                switch_off_ent_t = 0,
                switch_off_ent_r = 0,
                index=0 ):
        #print(self.__doc__) 
        self.yaxis_base = yaxis_base      
        self.t_in = t_profile    
        self.h_in = h_profile
        self.p_in = p_profile.astype('float')*100   # tansfer unit to Pa
        self.p_in[self.p_in<0] = np.nan
        self.z_in = z_profile   
        self.ind  = (~np.isnan(self.p_in)) & (~np.isnan(self.z_in)) &(np.array(self.p_in)>=10000.0)&(np.array(self.p_in)<=120000.0)
        self.z    = self.z_in[self.ind]
        self.t    = self.t_in[self.ind]
        if np.nanmean(self.t) < 100:
            self.t_in = t_profile+t0
            self.t = t0 + self.t       # covert from Celsius To Kelvin temperature
        self.t    = self.t_in[self.ind]
        self.h    = self.h_in[self.ind]
        self.p    = self.p_in[self.ind]
         
        self.td_r_q           = td_r_q  
            
        self.adiabat          =adiabat   
        # self.prc_w            =prc_w    
        self.qc_max           =qc_max   
        self.ice              =ice 
        ## parameter for newton      
        self.tice             =tice               
        self.newton_step_dt   =newton_step_dt 
        self.switch_off_ent_t = switch_off_ent_t
        self.switch_off_ent_r = switch_off_ent_r

        self.source = source
       
        self.X = X
        self.data_process()
        self.process() 
        #self.info()

    def info(self):      
        print('\tENTRAIN SCHEME:',entrainment_opt_list[self.entrain_opt][0])
        print('\tMOSITURE INPUT:',h_input_list[self.td_r_q][0])
        print('\tSOURCE:',source_list[self.source][0])
        print('\tPressure(Pa):',self.p[0])
        print('\tAltitude(m):',self.z[0])
        print('\tTemperature(K):',self.t[0])
        print('\tDewpoint(K):',self.td[0])
        print('\tMixingRatio(g/g):',self.q[0])
        print()
    #initialize output variables
    def data_process(self):
        #input is dewpoint
        if self.td_r_q == 1: 
            if np.nanmean(self.h) < 100:
                self.td = t0 + self.h       # covert from Celsius To Kelvin temperature
            else:
                self.td = self.h
            self.q = get_qvs(self.p,self.td) 

        #input is mixing ratio
        elif self.td_r_q == 2:
            self.q  = self.h/1000   #单位换算为g/g
            self.td = np.array(dewpoint_from_specific_humidity(specific_humidity=(self.q/(1+self.q)), 
                                                      temperature=self.t*units('K'), 
                                                      pressure=(self.p/100)* units('hPa')).to(units('K')))                
        #input is relative humidity
        elif self.td_r_q ==3:
            self.q = mixing_ratio_from_relative_humidity(relative_humidity=self.h,
                                                       temperature=self.t*units('K'),
                                                       pressure=(self.p/100)* units('hPa'))
            self.td = np.array(dewpoint_rh(self.t*units('K'),self.h).to(units('K')))
        #input is specific humidity
        else: 
            q = self.h/ 1000 #单位换算为g/g
            self.q = q / (1-q) 
            self.td = np.array(dewpoint_from_specific_humidity(self.q,self.t,self.p))
        
        #self.prof = parcel_profile(self.p*units('Pa'), self.t[0]*units('K'), self.td[0]*units('K')).to('degC')
        pi  = np.power((self.p/p00),rddcp)              #ratio between temperature and potential temperature
        self.tv  = self.t* (1+ reps*self.q)/(1+self.q)  #virtual temperature
        self.thv = self.tv / pi                         #virtual potential temperature
        # s   = get_entropy(p, t, q, np.zeros(len(q)), np.zeros(len(q)));   # entropy,why?
        #==============================================================================
        #find source parcel
        if  self.source == 0:     #surface parcel           
            self.kmax = 0
        elif self.source == 1:   #specified mixing layer
            if not self.yaxis_base:
              self.kmax = np.where(self.z >= self.z[0]+500)[0][0]
              pick = (self.z >= self.z[0]) & (self.z <= self.z[0]+1000)
            else:  
              self.kmax = np.where(self.p >= self.p[0]-50)[0][0]
              pick = (self.p <= self.self.p[0]) & (self.p >= self.p[0]-100)
            self.p[self.kmax] =  np.nanmean(self.p[pick])
            self.t[self.kmax] =  np.nanmean(self.t[pick])
            self.q[self.kmax] =  np.nanmean(self.q[pick])
        else:
            sys.stderr.write("Unknown value for source")
            raise SystemExit(1)
        self.X[:self.kmax] = 0
        self.init_output_vari()

    def init_output_vari(self):
        self.bp = np.full([len(self.p_in), 1] , np.nan)
        self.tp = np.full([len(self.p_in), 1] , np.nan)
        self.qp = np.full([len(self.p_in), 3] , np.nan)
        self.ql = np.full((len(self.p_in),),np.nan)       
        self.td_in = np.full([len(self.p_in), 1] , np.nan)
        #initialize another group of temperary variables
        self.bp0     = np.full([len(self.p),  1], np.nan)
        self.tp0     = np.full([len(self.p),  1], np.nan)
        self.qp0     = np.full([len(self.p),  3], np.nan)
        self.dbpdzs0 = np.full([len(self.p),  9], np.nan)
        self.dbpdze0 = np.full([len(self.p), 10], np.nan)
        self.dbpdzp0 = np.full([len(self.p),  2], np.nan)
    
    def process(self):
        #==============================================================================
        #define parcel properties at initial location    
        k   = self.kmax
        p2  = self.p[self.kmax]  # pressure
        t2  = self.t[self.kmax]  # temperature
        qv2 = self.q[self.kmax]  # mixing ratio
        ql2 = 0        # liquid mixing ratio
        qi2 = 0        # ice mixing ratio
        qt  = qv2      # environment mixing ratio
        b2  = 0        # buoyancy
        doit = True
        #==============================================================================
        #begin ascent of parcel
        nk            = len(self.p) 
        self.bp0[k]   = b2
        self.tp0[k]   = t2
        self.qp0[k,:] = [qv2,0,0]
        X_list = np.full((nk,),np.nan)
        # lcl_idx = 0
        while doit and k < nk-1 :
        # for k in range(kmax,nk-1):
            k = k+1
            #values from last step
            p1  = p2
            p2  = self.p[k]
            t1  = t2
            qv1 = qv2
            ql1 = ql2
            qi1 = qi2
        #    print(p1,p2,t1,t2)
            s1  = get_entropy(p1, t1, qv1, ql1, qi1)
        #    print('s1:',s1)
        #==============================================================================   
        # Isentropic process 
        # 牛顿法逼近
            i  = 0;
            not_converged = True
            while not_converged:  
                i  = i + 1
                if self.ice == 1 :
                    fliq = np.max([np.min([(t2-self.newton_step_dt-self.tice)/(t0-self.tice), 1]), 0]);
                    fice = 1 - fliq
                else:
                    fliq = 1
                    fice = 0
                #保证湿度不凭空增加以及冰相湿度不为负，两个公式在数值较小的情况下也会有此情况发生   
                qv2 = np.min([qt, fliq * get_qvs(p2, t2-self.newton_step_dt) + fice * get_qvi(p2, t2 - self.newton_step_dt)])
                qi2 = np.max([fice * (qt - qv2), 0])
                ql2 = np.max([qt - qv2 - qi2, 0])
                #print(p2, t2-newton_step_dt, qv2, ql2, qi2)
                fs2 = get_entropy(p2, t2-self.newton_step_dt, qv2, ql2, qi2) - s1
                
                if self.ice == 1 :
                    fliq = np.max([np.min([(t2-self.tice)/(t0-self.tice), 1]), 0]);
                    fice = 1 - fliq;
                else :
                    fliq = 1;
                    fice = 0;
                
                qv2 = np.min([qt, fliq * get_qvs(p2, t2) + fice * get_qvi(p2, t2)])
                qi2 = np.max([fice * (qt - qv2), 0])
                ql2 = np.max([qt - qv2 - qi2, 0])
                ##print(p2, t2, qv2, ql2, qi2)
                fs1 = get_entropy(p2, t2, qv2, ql2, qi2) - s1
                
                dt2 = self.newton_step_dt * fs1 / (fs2 - fs1) 
       
                #牛顿法迭代90次以上不收敛考虑调整参数       
                if i > 90:
                    print([k, self.z[k], t2, dt2])
                
                if i > 100:
                    print('lack of convergence, stopping iteration.')
                    break  ##需要确认
    
                if np.abs(dt2) > converge: 
                    t2  = t2 + dt2
                else:
                    ##收敛，气块温度计算结束，输出变量
                    not_converged = False;
                    tv2 = t2 * (1 + reps * qv2) / (1 + qt)    #virtual temperature
                    b2  = g * (tv2 - self.tv[k])/self.tv[k]
                    #store parcel temperature gradient related to different process
                    tbar  = 0.5 * (t1 + t2);
                    qvbar = 0.5 * (qv1 + qv2);
                    qlbar = 0.5 * (ql1 + ql2);
                    qibar = 0.5 * (qi1 + qi2);
                    lhv   = lv1 - lv2 * tbar;
                    lhs   = ls1 - ls2 * tbar;
                    Rm    = Rd + Rv * qvbar
                    #total specific heat at constant pressure
                    cpm   = cpd + cpv * qvbar + cpl * qlbar + cpi * qibar;
                    tebar = 0.5 * (self.t[k]+self.t[k-1])
            #==============================================================================   
            ## Entrainment process (no detrainment)           
            # backup values before entrainment
            if not self.X[k] == 0:
                t2_  = t2
                qv2_ = qv2
                ql2_ = ql2
                qi2_ = qi2
                qt_  = qt
                cpm_ = cpd + cpv*qv2_ + cpl*ql2_ + cpi*qi2_;
               
                # qt   = ( qt/(1+qt_) + self.X[k] * self.q[k]/(1+self.q[k])) / (1+self.X[k]) # environment mixing ratio
                # qv2m = (qv2/(1+qt_) + self.X[k] * self.q[k]/(1+self.q[k])) / (1+self.X[k]) # mixing ratio
                
                # ql2m = ql2/(1+qt_) / (1+self.X[k]);
                # qi2m = qi2/(1+qt_) / (1+self.X[k]);
                # qv2m = qv2m / (1-qt);  # specific humidity --> mixing ratio
                # ql2m = ql2m / (1-qt);   #qt is mixing ratio!!!!
                # qi2m = qi2m / (1-qt);
                # qt   = qt / (1-qt);
               
                # cpme = cpd + cpv*self.q[k]
                # cpmm = cpd + cpv*qv2m + cpl*ql2m + cpi*qi2m
                # t2m  = (1/(1+qt_)*cpm_*t2_ + 1/(1+self.q[k])*cpme*self.t[k]*self.X[k]) / cpmm*(1+qt) / (1+self.X[k])
                # t2md = (t2_ + self.X[k] * self.t[k]) / (1+self.X[k])
               
                if self.switch_off_ent_r:
                  qt   = (  qt/(1+qt_) + self.X[k] * qv2_/(1+qv2_)) / (1+self.X[k])
                  qv2m  = (qv2_/(1+qt_) + self.X[k] * qv2_/(1+qv2_)) / (1+self.X[k])
                else:
                  qt   = ( qt/(1+qt_) + self.X[k] * self.q[k]/(1+self.q[k])) / (1+self.X[k]) # environment mixing ratio
                  qv2m = (qv2/(1+qt_) + self.X[k] * self.q[k]/(1+self.q[k])) / (1+self.X[k]) # mixing ratio
                  
                ql2m = ql2/(1+qt_) / (1+self.X[k]);
                qi2m = qi2/(1+qt_) / (1+self.X[k]);
                qv2m = qv2m / (1-qt);  # specific humidity --> mixing ratio
                ql2m = ql2m / (1-qt);   #qt is mixing ratio!!!!
                qi2m = qi2m / (1-qt);
                qt   = qt / (1-qt);

                if self.switch_off_ent_t:
                  cpme = cpd + cpv*self.q[k]
                  cpmm = cpd + cpv*qv2m + cpl*ql2m + cpi*qi2m
                  t2m  = (1/(1+qt_)*cpm_*t2_ + 1/(1+self.q[k])*cpme*t2_*self.X[k]) / cpmm*(1+qt) / (1+self.X[k])
                  t2md = t2_
                elif self.switch_off_ent_r:
                  cpme = cpd + cpv*qv2m;
                  cpmm = cpd + cpv*qv2m + cpl*ql2m + cpi*qi2m
                  t2m  = (1/(1+qt_)*cpm_*t2_ + 1/(1+qv2m)*cpme*self.t[k]*self.X[k]) / cpmm*(1+qt) / (1+self.X[k])
                  t2md = (t2_ + self.X[k] * self.t[k]) / (1+self.X[k])
                else:
                  cpme = cpd + cpv*self.q[k]
                  cpmm = cpd + cpv*qv2m + cpl*ql2m + cpi*qi2m
                  t2m  = (1/(1+qt_)*cpm_*t2_ + 1/(1+self.q[k])*cpme*self.t[k]*self.X[k]) / cpmm*(1+qt) / (1+self.X[k])
                  t2md = (t2_ + self.X[k] * self.t[k]) / (1+self.X[k])

                t2 = t2m
                i = 0;
                not_converge = True;
                while not_converge:
                    i = i + 1;
                    #t = t2-1
                    if self.ice == 1:
                        fliq = np.max([np.min([(t2-self.newton_step_dt-self.tice)/(t0-self.tice), 1]), 0]);
                        fice = 1 - fliq;
                    else:
                        fliq = 1;
                        fice = 0;
                    
                    qv2 = np.min([qt, fliq*get_qvs(p2,t2-self.newton_step_dt) + fice*get_qvi(p2,t2-self.newton_step_dt)]);
                    qi2 = np.max([fice*(qt-qv2), 0]);
                    ql2 = np.max([qt-qv2-qi2, 0]);
                    cpm = cpd + cpv*qv2 + cpl*ql2 + cpi*qi2;
                    lhv = lv1 - lv2 * t2m;
                    lhs = ls1 - ls2 * t2m;
                    fs2 = 1/(1+qt) * cpm * (t2 - self.newton_step_dt - t2m ) + \
                          lhv * (qv2-qv2m) - (lhs-lhv) * (qi2-qi2m)
                    
                    if self.ice == 1:
                        fliq = np.max([np.min([(t2-self.tice)/(t0-self.tice), 1]), 0]);
                        fice = 1 - fliq;
                    else:
                        fliq = 1;
                        fice = 0;
                    
                    qv2 = np.min([qt, fliq*get_qvs(p2,t2) + fice*get_qvi(p2,t2)]);
                    qi2 = np.max([fice*(qt-qv2), 0]);
                    ql2 = np.max([qt-qv2-qi2, 0]);
                    cpm = cpd + cpv*qv2 + cpl*ql2 + cpi*qi2;
                    fs1 = 1/(1+qt)*cpm  * (t2 - t2m) + \
                        lhv * (qv2-qv2m) - (lhs-lhv) * (qi2-qi2m)  ###？？？？what's variable
                    dt2 = self.newton_step_dt * fs1 / (fs2-fs1)
                    
                    if i > 90:
                        print([i, t2, fs1, fs2, dt2])
                    
                    if i > 100:
                        print('lack of convergence, stopping iteration.')
                        break
                    
                    if np.abs(dt2) > converge:
                        t2 = t2 + dt2;
                    else:
                        not_converge = False
                        tv2 = t2 * (1 + reps * qv2) / (1 + qt) 
                        b2  = g * (tv2 - self.tv[k])/self.tv[k]    
            # if ql2 >0 and lcl_idx = 0:
            # 	lcl_idx = 1
            # 	self.lcl[k] = self.z[k]
            #==============================================================================
            # Precipitation process, change mp, qp (condensate fall out, no latent heat)
            ql2_ = ql2
            qi2_ = qi2
            self.ql[k] = ql2_
            # determine how many condensate fall out of parcel
            if self.adiabat > 0:        
                if self.adiabat == 1 :        #all condensate fall out               
                    ql2 = 0; 
                    qi2 = 0;
                elif self.adiabat == 2:     # max mixing ratio specified
                    f   = np.min([ql2+qi2, self.qc_max])/(ql2+qi2);
                    ql2 = f * ql2;
                    qi2 = f * qi2;
                # elif self.adiabat == 3:     # specific proportion
                #     ql2 = ql2 * self.prc_w * dz;
                #     qi2 = qi2 * self.prc_w * dz;
                else:
                    sys.stderr.write('Undefined adiabat')
                    raise SystemExit(1)
        #        m2  = (1 + qv2 + ql2 + qi2) / (1 + qt) * m2; % lose mass
                qt  = qv2 + ql2 + qi2;
                tv2 = t2 * (1 + reps*qv2) / (1 + qt)
                b2  = g  * (tv2 - self.tv[k])  / self.tv[k]
                #print(tv2,self.tv[k])

            # store final buoyancy and related variables
            self.bp0[k]    = b2;
            self.tp0[k]    = t2;
            self.qp0[k, :] = [qv2, ql2, qi2];  
             # stop if b < 0 and p < 100 hPa
            if  (self.p[k] <= 10000) & (b2 < 0): 
                doit = False
            
        # save output results
        self.bp[self.ind]             = self.bp0         # parcel buoyancy
        self.tp[self.ind]             = self.tp0         # parcel temperature
        self.qp[self.ind]             = self.qp0 * 1000  # mixing ratio of vapor,liquid,ice
        self.td_in[self.ind]          = np.reshape(self.td,(-1,1)) 
        #print(self.t.shape,self.bp0 .shape)
        self.td_in = np.squeeze(self.td_in)
        self.bp    = np.squeeze(self.bp)
        self.tp    = np.squeeze(self.tp)
        self.qp    = np.squeeze(self.qp)
        self.ind = self.ind & (~np.isnan(self.tp))

def plot_t(p_profile,t_profile,td_profile):
    from metpy.plots import SkewT
    #$ import matplotlib.pyplot as plt
    from matplotlib.ticker import MultipleLocator, FormatStrFormatter
    ind  = (~np.isnan(p_profile)) & (~np.isnan(z_profile)) &(np.array(p_profile)>=100.0)&(np.array(p_profile)<=1200.0)

    #def draw_skewT_buoyancy(p_in,T_in,Td_in,Tp_in=[],bp=[],index=0):        
    p  = ( p_profile/100) * units('hPa')
    T  = ( t_profile * units('K')).to(units('degC'))
    Td = (td_profile* units('K')).to(units('degC'))
    Tp = (np.squeeze(td_profile[ind])   * units('K')).to(units('degC'))
    #print(Tp.shape,T_in.shape)

    fig  = plt.figure(figsize=(15,9))
    skew = SkewT(fig,rotation=45,subplot=(1,2,1))
    
    # Plot the data using normal plotting functions, in this case using
    # log scaling in Y, as dictated by the typical meteorological plot
    skew.plot(p, T, 'r',label='Temperature')
    skew.plot(p, Td, 'g',label='Dew_T')
    # style_list=['-', '--', '-.', ':',]
    # label_list=['NoEntrain','ConstEntrain','DIA'] 
    skew.plot(p, Tp,'k',linestyle=':',label='Parcel_T')
    #skew.plot_barbs(p, u, v)
    skew.ax.set_ylim(1000, 100)
    skew.ax.set_xlim(-40, 60)
    #print(p[0], T[0], Td[0])
    # Calculate LCL height and plot as black dot
    lcl_pressure, lcl_temperature = lcl(p[0], T[0], Td[0])
    skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')
    print('LCL:',lcl_pressure, lcl_temperature)
    # Calculate full parcel profile and add to plot as black line
    prof = parcel_profile(p, T[0], Td[0]).to('degC')
    skew.plot(p, prof, 'k', linewidth=2,label='Ideal_Pacel_T')
    #plt.legend()
    
    #Shade areas of CAPE and CIN
    #skew.shade_cin(p, T, prof)
    #skew.shade_cape(p, T, prof)
    skew.shade_cin(p, Tp,T)
    skew.shade_cape(p,  Tp,T)
    
    # An example of a slanted line at constant T -- in this case the 0
    # isotherm
    skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)
    
    # Add the relevant special lines
    skew.plot_dry_adiabats()
    skew.plot_moist_adiabats()
    skew.plot_mixing_lines()
    plt.legend() 
    
   #  ax=fig.add_subplot(1,2,2)
   #  ymajorLocator = MultipleLocator(1) #将y轴主刻度标签设置为0.5的倍数
   #  ymajorFormatter = FormatStrFormatter('%d')
   #  ax.yaxis.set_major_locator(ymajorLocator)
   #  ax.yaxis.set_major_formatter(ymajorFormatter)

   #  plt.grid(linestyle=':',linewidth=2)
   #  plt.plot(bp,p,'k')
   #  ax.yaxis.grid(True,which='major')    
   #  plt.ylim([100,1000])
   # # ax.set_yticks(visible=False)
   #  ax.set_yscale('log')
   #  ax.invert_yaxis()
   #  plt.xlabel("Buoyancy(0.01m/${s^2}$)")    
   #  # Show the plot
   #  plt.show()


#%%========intergrate buoyancy based on height ========================================

class sounding_cal_byH():
    def __init__(self,height,press, temperature, dewpt, 
                 parcel_temperature_profile=None,
                 liquid_water  = None,  ##to cal LCL
                 specific_height_start=None,
                 specific_height_end=None,
                 low_alt = [0,]):
      self.pressure    = press.to(units('Pa'))   # m
      self.height      = height   # m
      self.temperature = temperature.to(units('K'))
      self.dewpt       = dewpt.to(units('K'))
      self.parcel_temperature_profile =parcel_temperature_profile.to(units('K'))
      self.liquid_water = liquid_water
      self.skip_index  = 0
      self.specific_height_start = specific_height_start #level for integrate,unit:m
      self.specific_height_end   = specific_height_end   #level for integrate
      self.low_alt = low_alt
      if parcel_temperature_profile is None:
          new_stuff = parcel_profile_with_lcl(self.pressure, self.temperature, self.dewpt)
          pressure, temperature, _, parcel_temperature_profile = new_stuff
          temperature = temperature.to('degC')
          self.parcel_temperature_profile = parcel_temperature_profile.to('degC')
      self.cal_el()
      self.cal_lfc()
      # if self.skip_index == 1:
      #   return
      self.integrate_buoyancy()
    def cal_lfc(self):
        self.lfc = data_struct()
        z, x = find_intersections(self.height[1:], self.parcel_temperature_profile[1:],
                              self.temperature[1:], direction='increasing')
        x, y = find_intersections(self.pressure[1:], self.parcel_temperature_profile[1:],
                              self.temperature[1:], direction='increasing')
        
        
        #=======================LFC====================================================
        # The LFC could:
        # 1) Not exist
        # 2) Exist but be equal to the LCL
        # 3) Exist and be above the LCL
        #print(x,y)
        # LFC does not exist or is LCL
        if len(x) == 0:
            self.lfc.p = np.nan * units('hPa') 
            self.lfc.t = np.nan * units('degC')
            self.lfc.h = np.nan * units('m')
        # LFC exists and is not LCL. Make sure it is above the LCL.
        else:
            if self.liquid_water is None:
              idx = x < lcl(self.pressure[0], self.temperature[0], self.dewpt[0])[0]
            else:             
              idx = x < self.pressure[self.liquid_water >0][0]
            print(len(x),idx)
            if sum(idx)>0:
                self.lfc.p = x[idx][-1]
                self.lfc.t = y[idx][-1]
                self.lfc.h = z[idx][-1]
            else:
                self.lfc.p = np.nan * units('hPa')
                self.lfc.t = np.nan * units('degC')
                self.lfc.h = np.nan * units('m') 
    def cal_el(self):
        self.el = data_struct()
        if self.parcel_temperature_profile[-1] > self.temperature[-1]:
            self.el.p = np.nan * self.pressure.units
            self.el.t = np.nan * self.temperature.units
        z, y = find_intersections(self.height[1:], self.parcel_temperature_profile[1:],
                              self.temperature[1:], direction='decreasing')
        x, y = find_intersections(self.pressure[1:], self.parcel_temperature_profile[1:],
                              self.temperature[1:], direction='decreasing')
        # Otherwise the last intersection (as long as there is one) is the EL     
        if len(x) > 0:
            self.el.p = x[-1]
            self.el.t = y[-1]  
            self.el.h = z[-1]      
        else:
            self.el.p = np.nan * units('hPa')
            self.el.t = np.nan * units('degC')
            self.el.h = np.nan * units('m')
       
    def integrate_buoyancy(self):  
        y = (self.parcel_temperature_profile - self.temperature).to(units.degK)
        x, y = _find_append_zero_crossings(np.copy(self.pressure), y)
        
        if (~np.isnan(self.lfc.p.magnitude)) & (~np.isnan(self.el.p.magnitude)):
          p_mask = _less_or_close(x, self.lfc.p.magnitude) & _greater_or_close(x,  self.el.p.magnitude)
          x_clipped = x[p_mask]
          y_clipped = y[p_mask]
          self.cape = (mpconsts.Rd
              * (np.trapz(y_clipped, np.log(x_clipped)) * units.degK)).to(units('J/kg'))
          
          #negative energy---cin like
          p_mask = _greater_or_close(x, self.lfc.p.magnitude)
          x_clipped = x[p_mask]
          y_clipped = y[p_mask]
          #print(x_clipped)
          self.cin  = (mpconsts.Rd
                 * (np.trapz(y_clipped, np.log(x_clipped)) * units.degK)).to(units('J/kg'))
        else:
          self.cape = np.nan*units('J/kg')
          self.cin = np.nan*units('J/kg')
        #==============calculate integrate buoyance of different level==================================
        y = (self.parcel_temperature_profile - self.temperature).to(units.degK) 
        x = self.pressure.magnitude
        if (self.specific_height_start == None) & (self.specific_height_end == None) : 
          self.height_list  = ([0,1000,2000,3000,4000,5000,6000,8000,10000,12000,14000,16000])*units.m
          self.integ_b = []
          for i in range(len(self.height_list)-1):
            start_level = self.low_alt[0]+self.height_list[i]
            end_level   = self.low_alt[0]+self.height_list[i+1]
            p_mask = _less_or_close(self.height, end_level) & _greater_or_close(self.height, start_level)
            x_clipped = x[p_mask]
            y_clipped = y[p_mask]
            self.integ_b = np.append(self.integ_b,(mpconsts.Rd*(np.trapz(y_clipped[::-1], np.log(x_clipped)[::-1]) * units.degK)).to(units('J/kg')))
        else:
            start_level = self.specific_height_start
            end_level   = self.specific_height_end
            p_mask = _less_or_close(self.height, end_level) & _greater_or_close(self.height, start_level)
            x_clipped = x[p_mask]
            y_clipped = y[p_mask]
            #print(x_clipped,y_clipped)
            self.integ_b = (mpconsts.Rd*(np.trapz(y_clipped[::-1], np.log(x_clipped[::-1])) * units.degK)).to(units('J/kg'))

def plot_t(p_profile,t_profile,td_profile):
    from metpy.plots import SkewT
    #$ import matplotlib.pyplot as plt
    from matplotlib.ticker import MultipleLocator, FormatStrFormatter
    ind  = (~np.isnan(p_profile)) & (~np.isnan(z_profile)) &(np.array(p_profile)>=100.0)&(np.array(p_profile)<=1200.0)

    #def draw_skewT_buoyancy(p_in,T_in,Td_in,Tp_in=[],bp=[],index=0):        
    p  = ( p_profile/100) * units('hPa')
    T  = ( t_profile * units('K')).to(units('degC'))
    Td = (td_profile* units('K')).to(units('degC'))
    Tp = (np.squeeze(td_profile[ind])   * units('K')).to(units('degC'))
    #print(Tp.shape,T_in.shape)

    fig  = plt.figure(figsize=(15,9))
    skew = SkewT(fig,rotation=45,subplot=(1,2,1))
    
    # Plot the data using normal plotting functions, in this case using
    # log scaling in Y, as dictated by the typical meteorological plot
    skew.plot(p, T, 'r',label='Temperature')
    skew.plot(p, Td, 'g',label='Dew_T')
    # style_list=['-', '--', '-.', ':',]
    # label_list=['NoEntrain','ConstEntrain','DIA'] 
    skew.plot(p, Tp,'k',linestyle=':',label='Parcel_T')
    #skew.plot_barbs(p, u, v)
    skew.ax.set_ylim(1000, 100)
    skew.ax.set_xlim(-40, 60)
    #print(p[0], T[0], Td[0])
    # Calculate LCL height and plot as black dot
    lcl_pressure, lcl_temperature = lcl(p[0], T[0], Td[0])
    skew.plot(lcl_pressure, lcl_temperature, 'ko', markerfacecolor='black')
    print('LCL:',lcl_pressure, lcl_temperature)
    # Calculate full parcel profile and add to plot as black line
    prof = parcel_profile(p, T[0], Td[0]).to('degC')
    skew.plot(p, prof, 'k', linewidth=2,label='Ideal_Pacel_T')
    #plt.legend()
    
    #Shade areas of CAPE and CIN
    #skew.shade_cin(p, T, prof)
    #skew.shade_cape(p, T, prof)
    skew.shade_cin(p, Tp,T)
    skew.shade_cape(p,  Tp,T)
    
    # An example of a slanted line at constant T -- in this case the 0
    # isotherm
    skew.ax.axvline(0, color='c', linestyle='--', linewidth=2)
    
    # Add the relevant special lines
    skew.plot_dry_adiabats()
    skew.plot_moist_adiabats()
    skew.plot_mixing_lines()
    plt.legend() 

@njit
def lfc(pressure, temperature, dewpt, parcel_profile):
    x, y = find_intersections(pressure[1:], parcel_profile[1:],
                              temperature[1:], direction='increasing')

    # The LFC could:
    # 1) Not exist
    # 2) Exist but be equal to the LCL
    # 3) Exist and be above the LCL

    # LFC does not exist or is LCL
    if len(x) == 0:
        if sum(_less_or_close( temperature,parcel_profile))<3:
            # LFC doesn't exist
            return np.nan * pressure.units, np.nan * temperature.units
        else:  # LFC = LCL
            x, y = lcl(pressure[0], temperature[0], dewpt[0])
            return x, y

    # LFC exists and is not LCL. Make sure it is above the LCL.
    else:
        idx = x < lcl(pressure[0], temperature[0], dewpt[0])[0]
        x = x[idx]
        y = y[idx]
        if len(x)> 0:  
            return x[0], y[0]
        else:
            return np.nan * pressure.units, np.nan * temperature.units
    
def cape_cin(pressure, temperature, dewpt, parcel_profile):
    #print(parcel_profile.shape,pressure.shape, temperature.shape, dewpt.shape)
    lfc_pressure, _ = lfc(pressure, temperature, dewpt,parcel_profile)
    # If there is no LFC, no need to proceed.
    if np.isnan(lfc_pressure):
        return 0 * units('J/kg'), 0 * units('J/kg'),np.nan * units('hPa'), np.nan * units('hPa')
    else:
        lfc_pressure = lfc_pressure.magnitude
    # Calculate the EL limit of integration
    el_pressure, _ = el(pressure, temperature, dewpt, parcel_profile)

    # No EL and we use the top reading of the sounding.
    if np.isnan(el_pressure):
        el_pressure = 100
    else:
        el_pressure = el_pressure.magnitude
    # Difference between the parcel path and measured temperature profiles
    y = (parcel_profile - temperature).to(units.degK)

    # Estimate zero crossings
    x, y = _find_append_zero_crossings(np.copy(pressure), y)

    # CAPE
    # Only use data between the LFC and EL for calculation
    p_mask = _less_or_close(x, lfc_pressure) & _greater_or_close(x, el_pressure)
    x_clipped = x[p_mask]
    y_clipped = y[p_mask]
    cape = (mpconsts.Rd
            * (np.trapz(y_clipped, np.log(x_clipped)) * units.degK)).to(units('J/kg'))

    # CIN
    # Only use data between the surface and LFC for calculation
    p_mask = _greater_or_close(x, lfc_pressure)
    x_clipped = x[p_mask]
    y_clipped = y[p_mask]
    cin = (mpconsts.Rd
           * (np.trapz(y_clipped, np.log(x_clipped)) * units.degK)).to(units('J/kg'))
#    print(cape,cin)
    
    return cape,cin, lfc_pressure, el_pressure       
#=================calculate buoyance from Neelin=====================================================================================
"""
Entraining plume buoyancy calculation utility functions
    Yi-Hung Kuo, 2019/10/3
    (Adopted from scripts Fiaz Ahmed)

Units:
    SI (temperature in K).

Notations:
    r,rl,ri,rt: mixing ratio of water (vapor,liquid,ice,total).
    q,ql,qi,qt: specific humidity/liquid/ice/total water content.

- The following are functions defined by formulae:
    Tv: virtual temperature (or density temperature).
    
    es: saturation vapor pressure with respect to liquid water.
    es_bolton: es from Bolton (1980).
    esi: saturation vapor pressure over ice.

    qs: saturation specific humidity with respect to liquid water.
    qsi: saturation specific humidity over ice.
    
    theta_v: virtual potential temperature.
    theta_e: equivalent potential temperature.
    theta_l: liquid water potential temperature.
    theta_il: ice-liquid potential temperature. 

Main function:
    cal_plume_buoyancy
    - input: T_env, q_env, X, plu_init_lev, p.
"""
# w = np.sin(np.pi*(h-z_1000hPa)/1.4e4)
# w[h-z_1000hPa>1.4e4] = 0.
# w[p>=1e5] = 0.
# omg = conv_w2omega(w, T_env, q_env, 0., p)
# # Calculate mixing coefficient from omega
# X_DIB = np.divide(omg[1:]-omg[0:-1], omg[1:], out=np.full(omg.shape[0]-1, np.nan), where=omg[1:]!=0.)
# X_DIB = np.hstack((X_DIB, 0.))
# X_DIB[X_DIB<0.] = 0. # 0 above min omega (~max w)


import numpy as np
from math import exp,log,sqrt,pow
from numba import njit
#import timeit

"""
Thermodynamic constants used for plume buoyancy calculations.

Notations (in lowercase) & values from Appendix 2 (p. 566-567) of Emanuel (1994).

Ref: Emanuel, K. A. 1994. Atmospheric Convection. Oxford University Press, 580 pp. 
"""

# Units: J/Kg/K
rd = 287.04 # gas constant of dry air
rv = 461.50 # gas constant of water vapor
cvd = 719. # heat capacity at constant volume for dry air
cpd = 1005.7 # heat capacity at constant pressure for dry air
cvv = 1410. # heat capacity at constant volume of water vapor
cpv = 1870. # heat capacity at constant pressure of water vapor
cl = 4190. # heat capacity of liquid water (above freezing)
cpvmcl = cl-cpv # cpvmcl seems to be a common notation for this value

# Units: dimensionless
epsilon = rd/rv

# Units: J/Kg
lv0 = 2.501E6 # latent heat of vaporization at 0-deg-C
ls = 2.834E6 # latent heat of sublimation (-100<=T<= 0-deg-C)
lf = 0.3337E6 # latent heat of fusion at 0-deg-C (lv0-ls)

# Units: m/s^2
g = 9.80665 # standard gravity



# plume_lifting_erai in Fiaz's code (modified).
@njit
def cal_plume_buoyancy(T_env, q_env,
                          X, plu_init_lev, p,
                            qc_cap=1e-3,
                          RV=rv,
                          CPD=cpd,
                          CL=cl,
                          LV0=lv0,
                          LF=lf):
    """Entraining plume calculation, assuming pressure decreasing with index.
    
    Input: environment temperature & humidity profiles,
    entrainment assumption, and initial plume level.
    
    Output: plume virtual temperature, temperature, specific humidity & total water content profiles.
      and saturation specific humidity.
    
    Plume calculation: given a conservative variable r_env of the environment, 
    the corresponding value in a entraining plume r_plu is determined by
    
        (d/dz) r_plu = epsilon * (r_env - r_plu) ... (*) [Eq. (5) in Holloway & Neelin (2009)].
        
    epsilon: fractional entrainment rate (units: 1/m) with respect to and as a function of height z.
    
    In this subroutine, Eq. (*) is integrated with respect to z using forward Euler method, i.e.,
    
        r_plu_k = [1 - X_(k-1)] * r_plu_(k-1) + X_(k-1) * r_env_(k-1) 
                                                 ... (**) [Eq. (1) in schiro et al. (2016)].
                                                
    _k: values at k-th level (in pressure p or height z).
    X: mixing coefficient = epsilon * dz_k
                          = epsilon * (dz_k/dp_k) * dp_k
                          ~ - epsilon *(1/rho/g) * dp_k.
    dz_k: z_k - z_(k-1).
    dp_k: p_k - p_(k-1).
    rho: air density.
    
    Note: the mixing coefficient X is identical to the input variable X.
    
    Reference:
        Holloway, C. E., and J. D. Neelin, 2009: Moisture vertical structure, column water vapor,
            and tropical deep convection. J. Atmos. Sci., 66, 1665-1683.
        Schiro, K. A., J. D. Neelin, D. K. Adams, and B. R. Lintner, 2016: Deep convection and 
            column water vapor over tropical land versus tropical ocean: A comparison between
            the Amazon and the tropical western Pacific. J. Atmos. Sci., 73, 4043-4063.
        
    Args:
        Tv_plu:   ndarray(dtype=float, ndim=1)
            Place holder for outputing plume virtual temperature profile in K.
        T_plu:    ndarray(dtype=float, ndim=1)
            Place holder for outputing plume temperature profile in K.
        q_plu:    ndarray(dtype=float, ndim=1)
            Place holder for outputing plume specific humidity profile in Kg/Kg. 
        qt_plu:   ndarray(dtype=float, ndim=1)
            Place holder for outputing plume total water content profile in Kg/Kg. 
        Tv_env:   ndarray(dtype=float, ndim=1)
            Place holder for outputing environment virtual temperature profile in K.

        T_env:    ndarray(dtype=float, ndim=1)
            Environment temperature profile in K.
        q_env:    ndarray(dtype=float, ndim=1)
            Environment specific humidity profile in Kg/Kg.
        qsat_env: ndarray(dtype=float, ndim=1)
            Place holder for environment saturation specific humidity (liquid/ice depends on T_env) in Kg/Kg.
        theta_l_env: ndarray(dtype=float, ndim=1)
            Place holder for environment equivalent potential temperature (liquid) in K.
        theta_l_sat_env: ndarray(dtype=float, ndim=1)
            Place holder for environment saturation equivalent potential temperature (liquid) in K.
        X    :    ndarray(dtype=float, ndim=1)
            mixing coefficient (dimensionaless).
        plu_init_lev (int):
            Index of the pressure level at which the plume originates.
        p    :    ndarray(dtype=float, ndim=1)
            Pressure levels in Pa. 
        qc_capped:ndarray(dtype=float, ndim=1)
            Place holder for outputing profile of amount of capped condensate in Kg/Kg.
        qc_cap (double): cap of plume water condensates in Kg/Kg.
        RV (constant): gas constant of water vapor in J/Kg/K.
        CPD (constant): heat capacity at constant pressure for dry air in J/Kg/K.
        CL (constant): heat capacity of liquid water (above freezing) in J/Kg/K
        LV0 (constant): latent heat of vaporization at 0-deg-C in J/Kg
        LF (constant): latent heat of fusion at 0-deg-C in J/Kg
    """
    num_lev = T_env.shape[0]
    Tv_plu = np.full(shape = T_env.shape,fill_value = np.nan) 
    T_plu = np.full(shape = T_env.shape,fill_value = np.nan) 
    q_plu = np.full(shape = T_env.shape,fill_value = np.nan) 
    qt_plu = np.full(shape = T_env.shape,fill_value = np.nan)
    Tv_env = np.full(shape = T_env.shape,fill_value = np.nan) 
    qsat_env = np.full(shape = T_env.shape,fill_value = np.nan)
    theta_l_env = np.full(shape = T_env.shape,fill_value = np.nan) 
    theta_l_sat_env = np.full(shape = T_env.shape,fill_value = np.nan) 
    qc_capped= np.full(shape = T_env.shape,fill_value = np.nan) 
    
    theta_il_env = np.zeros(num_lev)
    theta_il_plu = np.zeros(num_lev)
    # cal liquid-ice equivalent potential temperature
    for lev_idx in np.arange(plu_init_lev,num_lev):
        theta_il_env[lev_idx] = eval_theta_il(T_env[lev_idx], 
                                              q_env[lev_idx], 
                                              0., 0., p[lev_idx])

    freezing = 0 # initialization; change to 1 if temperature falls below freezing
    
    theta_il_plu[plu_init_lev] = theta_il_env[plu_init_lev]
    qt_plu[plu_init_lev] = q_env[plu_init_lev]
    # get air temperature and specific humidity from potential temperature and dewpoint 
    T_plu[plu_init_lev], q_plu[plu_init_lev] = inv_T_from_theta_l(theta_il_plu[plu_init_lev], 
                                                                  qt_plu[plu_init_lev], 
                                                                  p[plu_init_lev])
    
    
    for lev_idx in np.arange(plu_init_lev+1, num_lev):
        #print(lev_idx)
        # Mixing liquid water potential temperature theta_l and total water content qt
        if X[lev_idx-1]>0.:
            theta_il_plu[lev_idx] = theta_il_plu[lev_idx-1]*(1.-X[lev_idx-1]) + theta_il_env[lev_idx-1]*X[lev_idx-1]
            qt_plu[lev_idx] = qt_plu[lev_idx-1]*(1.-X[lev_idx-1]) + q_env[lev_idx-1]*X[lev_idx-1]
        else:
            theta_il_plu[lev_idx] = theta_il_plu[lev_idx-1]
            qt_plu[lev_idx] = qt_plu[lev_idx-1]
        
        # Initialize qc_capped (amount of condensate removed by qc_cap)
        qc_capped[lev_idx] = qt_plu[lev_idx]
        
        if (np.isfinite(theta_il_plu[lev_idx]) and np.isfinite(qt_plu[lev_idx])):
            
            if freezing == 0:
                T_plu[lev_idx], q_plu[lev_idx] = inv_T_from_theta_l(theta_il_plu[lev_idx], 
                                                                    qt_plu[lev_idx], 
                                                                    p[lev_idx])
                
                if T_plu[lev_idx]<=273.15: # if temperature falls below freezing
                    # Convert liquid water to ice in one irreversible step (Emanuel 1994, pp. 139)
                    
                    #Calculate saturation specific humidity
                    QS = eval_qsat(T_plu[lev_idx],p[lev_idx])
                    
                    if qt_plu[lev_idx]<QS:
                        q_plu[lev_idx] = qt_plu[lev_idx]
                    else:
                        q_plu[lev_idx] = QS
                    
                    QL = qt_plu[lev_idx] - q_plu[lev_idx]
                    RL = QL / (1. - qt_plu[lev_idx])
                    RS = QS / (1. - qt_plu[lev_idx])
                    RT = RL+RS
                    alpha = 0.009705  # linearized e#/e* around 0-deg-C (to -1C)
                    bet = eval_esi(T_plu[lev_idx]) / eval_es(T_plu[lev_idx])
                    A = (LV0+LF) * alpha * LV0 * RS / RV / pow(T_plu[lev_idx], 2.)
                    B = CPD + CL*RT + alpha*(LV0 + LF)*RS + bet*(LV0+LF)*LV0*RS/RV/pow(T_plu[lev_idx], 2.)
                    C = -LV0*RS - LF*RT + bet*(LV0+LF)*RS
                    dT = (-B + sqrt(pow(B, 2.) - 4.*A*C)) / (2.*A)
                    T_plu[lev_idx] += dT
                    
                    QSI = eval_qsi(T_plu[lev_idx], p[lev_idx])
                    
                    if qt_plu[lev_idx]<QSI:
                        q_plu[lev_idx] = qt_plu[lev_idx]
                    else:
                        q_plu[lev_idx] = QSI
                        
                    QI = qt_plu[lev_idx] - q_plu[lev_idx]
                    theta_il_plu[lev_idx] = eval_theta_il(T_plu[lev_idx], q_plu[lev_idx], 0., QI, p[lev_idx])
                    Tv_plu[lev_idx] = eval_Tv(T_plu[lev_idx], q_plu[lev_idx], QI)
                    Tv_env[lev_idx] = eval_Tv(T_env[lev_idx], q_env[lev_idx], 0.)
                    qsat_env[lev_idx] = eval_qsat(T_env[lev_idx], p[lev_idx])
                    
                    freezing = 1
                
                else: # if temperature still above freezing
                    QS = eval_qsat(T_plu[lev_idx],p[lev_idx])
                    
                    if qt_plu[lev_idx]<QS:
                        q_plu[lev_idx] = qt_plu[lev_idx]
                    else:
                        q_plu[lev_idx] = QS
                        
                    QL = qt_plu[lev_idx] - q_plu[lev_idx]
                    # Top out liquid water content at qc_cap-Kg/Kg (default: 1e-3)
                    if QL>qc_cap:
                        qt_plu[lev_idx] -= QL-qc_cap
                        theta_il_plu[lev_idx] = eval_theta_il(T_plu[lev_idx], q_plu[lev_idx], qc_cap, 0., p[lev_idx])
                        QL = qc_cap
                        
                    Tv_plu[lev_idx] = eval_Tv(T_plu[lev_idx], q_plu[lev_idx], QL)
                    Tv_env[lev_idx] = eval_Tv(T_env[lev_idx], q_env[lev_idx], 0.)
                    qsat_env[lev_idx] = eval_qsat(T_env[lev_idx], p[lev_idx])
            
            else: # if freezing==1, i.e., already above freezing level
                T_plu[lev_idx], q_plu[lev_idx] = inv_T_from_theta_il(theta_il_plu[lev_idx], 
                                                                     qt_plu[lev_idx], 
                                                                     p[lev_idx])
                
                QI = qt_plu[lev_idx] - q_plu[lev_idx]
                # Top out liquid water content at qc_cap-Kg/Kg (default: 1e-3)
                if QI>qc_cap:
                    qt_plu[lev_idx] -= QI-qc_cap
                    theta_il_plu[lev_idx] = eval_theta_il(T_plu[lev_idx], q_plu[lev_idx], 0., qc_cap, p[lev_idx])
                    QI = qc_cap
                #cal Tv by temperature, water vapor content, liquid water content  
                Tv_plu[lev_idx] = eval_Tv(T_plu[lev_idx], q_plu[lev_idx], QI)
                Tv_env[lev_idx] = eval_Tv(T_env[lev_idx], q_env[lev_idx], 0.)
                qsat_env[lev_idx] = eval_qsat(T_env[lev_idx], p[lev_idx])
        
        # Finalize qc_capped (amount of condensate removed by qc_cap)
        qc_capped[lev_idx] -= qt_plu[lev_idx]
    
    for lev_idx in np.arange(plu_init_lev,num_lev):
        theta_l_env[lev_idx] = eval_theta_e(T_env[lev_idx], q_env[lev_idx], 0., p[lev_idx])
        theta_l_sat_env[lev_idx] = eval_theta_e(T_env[lev_idx], qsat_env[lev_idx], 0., p[lev_idx])
    return Tv_plu, T_plu, q_plu, qt_plu,Tv_env, qsat_env,theta_l_env, theta_l_sat_env, qc_capped
#==============================================================================================================
#=================================== additional function ======================================================
@njit
def ci(T):
    """Calculate heat capacity of ice.
    
    Units: J/Kg/K.
    
    Args:
        T (double): air temperature in K.
        
    Returns:
        double: heat capacity of ice at temperature T following Appendix 2 (p. 566-567) of Emanuel (1994).
    
    """
    return 2106. + 7.3*(T-273.15)

@njit
def lv(T):
    """Calculate latent heat of vaporization.
    
    Units: J/Kg.
    
    Args:
        T (double): air temperature in K.
        
    Returns:
        double: latent heat of vaporization at temperature T following Appendix 2 (p. 566-567) of Emanuel (1994).
    
    """
    return lv0 - cpvmcl*(T-273.15)
# temp_i_calc in Fiaz's code.
@njit
def inv_T_from_theta_il(theta_il, qt, p,
                        RD=rd,
                        RV=rv,
                        CPD=cpd,
                        LS=ls,
                        EPS=epsilon,
                        P0=1e+05,
                        ABS_TOL=1e-3): # add MAX_ITER???
    """Invert air temperature and specific humidyt, 
    given liquid-ice water potential temperature, total specific water content, pressure
    and assuming all condensates are ice.
    
    Units: K.
    
    Args:
        theta_il (double): liquid-ice water potential temperature in K.
        qt (double): specific total water content in kg/kg.
        p (double): air pressure in Pa.
        RD (constant): gas constant of dry air in J/Kg/K.
        RV (constant): gas constant of water vapor in J/Kg/K.
        CPD (constant): heat capacity at constant pressure for dry air in J/Kg/K.
        LS (constant): latent heat of sublimation in J/Kg/K (-100<=T<= 0-deg-C)
        EPS (constant): ratio of gas constant of dry air to gas constant of water vapor.
        P0 (constant): reference pressure (1,000 hPa) in Pa.
        ABS_TOL (constant): absolute tolerance (units: K) for stopping iteration.
        
    Returns:
        double: air temperature.
    
    """
    # Initialization for iteration
    dT = 999. # iteration stops when dT<ABS_TOL
    
    rt = qt / (1.-qt)
    
    TL = theta_il * pow(p/P0, RD/CPD)
    QS = eval_qsi(TL, p)
    rs = QS / (1.-qt)
    
    if (qt>QS):
        qi = qt-QS
        ri = rt-rs
    else:
        qi = 0.
        ri = 0.
    
    T = TL
    
    while (abs(dT)>=ABS_TOL):
        S = LS/CPD/T
        dT = (T - TL*(1.+S*ri)) / (1. + S*TL*(ri/T + (1.+rs*EPS)*rs*LS/(RV*T*T)))
        T = T-dT
        QS = eval_qsi(T,p)
        rs = QS / (1.-qt)
        
        if (qt>QS):
            q = QS
            qi = qt-QS
            ri = rt-rs
        else:
            q = qt
            qi = 0.
            ri = 0.
    
    # Constraint final theta_il using inital theta_il
    theta_il_final = eval_theta_il(T, q, 0., qi, p) # ql=0
    d_theta_il = theta_il_final-theta_il
   
    while (abs(d_theta_il)>0.05):
        if (abs(d_theta_il)<1.):
            dT = ABS_TOL * d_theta_il / abs(d_theta_il)
        else:
            dT = d_theta_il/3.
            
        T = T-dT
        QS = eval_qsi(T,p)
        
        if (qt>QS):
            q = QS
            qi = qt-QS
        else:
            q = qt
            qi = 0.
            
        theta_il_final = eval_theta_il(T, q, 0., qi, p) # ql=0
        d_theta_il = theta_il_final-theta_il
        
    return T, q

# temp_calc in Fiaz's code.
@njit # ~85 mju-s vs 3.5 mju-s without @njit
def inv_T_from_theta_l(theta_l, qt, p, 
                       RD=rd,
                       RV=rv,
                       CPD=cpd,
                       EPS=epsilon,
                       P0=1e+05,
                       ABS_TOL=1e-3): # add MAX_ITER???
    """Invert air temperature and specific humidity, 
    given liquid water potential temperature, total specific water content, and pressure.
    
    Units: K.
    
    Args:
        theta_l (double): liquid water potential temperature in K.
        qt (double): specific total water content in kg/kg.
        p (double): air pressure in Pa.
        RD (constant): gas constant of dry air in J/Kg/K.
        RV (constant): gas constant of water vapor in J/Kg/K.
        CPD (constant): heat capacity at constant pressure for dry air in J/Kg/K.
        EPS (constant): ratio of gas constant of dry air to gas constant of water vapor.
        P0 (constant): reference pressure (1,000 hPa) in Pa.
        ABS_TOL (constant): absolute tolerance (units: K) for stopping iteration.
        
    Returns:
        double: air temperature.
        double: specific humidity.
    
    """
    # Initialization for iteration
    dT = 999. # iteration stops when dT<ABS_TOL
    
    rt = qt / (1.-qt)
    
    TL = theta_l * pow(p/P0, RD/CPD)
    QS = eval_qsat(TL, p)
    rs = QS / (1.-qt)
    
    if (qt>QS):
        ql = qt-QS
        rl = rt-rs
    else:
        ql = 0.
        rl = 0.
        
    T = TL
    
    while (abs(dT)>=ABS_TOL):
        LV = lv(T)
        S = LV/CPD/T
        dT = (T - TL*(1.+S*rl)) / (1. + S*TL*(rl/T + (1.+rs*EPS)*rs*LV/(RV*T*T)))
        T = T-dT
        QS = eval_qsat(T,p)
        rs = QS / (1.-qt)
        
        if (qt>QS):
            q = QS
            ql = qt-QS
            rl = rt-rs
        else:
            q = qt
            ql = 0.
            rl = 0.
    
    # Constraint final theta_l using inital theta_l
    theta_l_final = eval_theta_l(T, q, ql, p)
    d_theta_l = theta_l_final-theta_l
   
    while (abs(d_theta_l)>0.05):
        if (abs(d_theta_l)<1.):
            dT = ABS_TOL * d_theta_l / abs(d_theta_l)
        else:
            dT = d_theta_l/3.
            
        T = T-dT
        QS = eval_qsat(T,p)
        
        if (qt>QS):
            q = QS
            ql = qt-QS
        else:
            q = qt
            ql = 0.
            
        theta_l_final = eval_theta_l(T, q, ql, p)
        d_theta_l = theta_l_final-theta_l
        
    return T, q

# theta_e_calc (eq. potential temperature) in Fiaz's code.
@njit
def eval_theta_ep(T, q, p,
                  EPS=epsilon,
                  P0=1e+05):
    """Calculate pseudoequivalent potential temperature.
    
    Units: K.
    
    Original from Bolton (1980) with curve-fitting.
    
    The saturation temperature TL [T* in Emanuel (1994), Eq. (4.6.24)] is used.
    TL represents the temperature at the LCL following an adiabat.
    Accuracy of TL within 0.3-deg-C in the range of atmospheric conditions.
    
    Args:
        T (double): air temperature in K.
        q (double): specific humidity of water vapor in kg/kg.
        p (double): air pressure in Pa.
        EPS (constant): ratio of gas constant of dry air to gas constant of water vapor.
        P0 (constant): reference pressure (1,000 hPa) in Pa.
        
    Returns:
        double: pseudoequivalent potential temperature following Eq. (4.7.9) in Emanuel (1994).
    """
    r = q / (1.-q)
    
    e_hPa = p * r / (EPS+r) / 100. # vapor pressure in hPa
    TL = 2840. / ( 3.5*log(T)-log(e_hPa)-4.805 ) + 55.
    
    return (T * pow(P0/p, 0.2854*(1.-0.28*r)) 
            * exp( r * (1.+0.81*r) * (3376./TL-2.54)  ) )  

@njit
def eval_theta_e(T, q, ql, p,
                 RD=rd,
                 RV=rv,
                 CPD=cpd,
                 CL=cl,
                 EPS=epsilon,
                 P0=1e+05):
    """Calculate equivalent potential temperature.
    
    Units: K.
    
    Follows Eq. (4.5.11) in Emanuel (1994),
        
    Args:
        T (double): air temperature in K.
        q (double): specific humidity of water vapor in kg/kg.
        ql (double): specific liquid water content in kg/kg.
        p (double): air pressure in Pa.
        RD (constant): gas constant of dry air in J/Kg/K.
        RV (constant): gas constant of water vapor in J/Kg/K.
        CPD (constant): heat capacity at constant pressure for dry air in J/Kg/K.
        CL (constant): heat capacity of liquid water (above freezing) in J/Kg/K.
        EPS (constant): ratio of gas constant of dry air to gas constant of water vapor.
        P0 (constant): reference pressure (1,000 hPa) in Pa.
        
    Returns:
        double: equivalent potential temperature.
    
    """
    LV = lv(T)
    
    r = q / (1.-q-ql)
    rt = (q+ql) / (1.-q-ql)
    
    e = p*q / ( EPS+(1.-EPS)*q )
    es = eval_es(T)
    rh = e/es
    pd = p-e
    
    chi = RD / (CPD + rt*CL)
    gamma = -r*RV / (CPD + rt*CL)
    
    if rt>0.:
        return (T * pow(P0/pd, chi)
                * pow(rh, gamma)
                * exp(LV*r / (CPD+rt*CL) / T) )
    else:
        return (T * pow(P0/pd, chi)
                * exp(LV*r / (CPD+rt*CL) / T) )

@njit
def eval_theta_il(T, q, ql, qi, p,
                  RD=rd,
                  RV=rv,
                  CPD=cpd,
                  CPV=cpv,
                  LS=ls,
                  EPS=epsilon,
                  P0=1e+05):
    """Calculate ice-liquid water equivalent potential temperature.
    
    Units: K.
    
    This is the CONSERVED variable in the entraining plume calculation.
    
    The formula is similar to Eq. (4.5.15) in Emanuel (1994),
    with Lv*rl replaced by Lv*rl+Ls*ri, and rl by rl+rt.
    The derivation is identical to the derivation of Eq. (4.5.15) on pp. 121,
    but adding ice phase into consideration.
    
    Args:
        T (double): air temperature in K.
        q (double): specific humidity of water vapor in kg/kg.
        ql (double): specific liquid water content in kg/kg.
        qi (double): specific ice content in kg/kg.
        p (double): air pressure in Pa.
        RD (constant): gas constant of dry air in J/Kg/K.
        RV (constant): gas constant of water vapor in J/Kg/K.
        CPD (constant): heat capacity at constant pressure for dry air in J/Kg/K.
        CPV (constant): heat capacity at constant pressure for water vapor in J/Kg/K.
        LS (constant): latent heat of sublimation in J/Kg/K (-100<=T<= 0-deg-C)
        EPS (constant): ratio of gas constant of dry air to gas constant of water vapor.
        P0 (constant): reference pressure (1,000 hPa) in Pa.
        
    Returns:
        double: ice-liquid water equivalent potential temperature.
    
    """
    LV = lv(T)
    
    r = q / (1.-q-ql-qi)
    rl = ql / (1.-q-ql-qi)
    ri = qi / (1.-q-ql-qi)
    rt = r+rl+ri
    
    chi = (RD + rt*RV) / (CPD + rt*CPV)
    gamma = rt*RV / (CPD + rt*CPV)
    
    if rt>0.:
        return (T * pow(P0/p, chi) * pow(1.-(rl+ri)/(EPS+rt), chi)
                * pow(1.-(rl + ri)/rt, -gamma)
                * exp((-LV*rl-LS*ri) / (CPD+rt*CPV) / T) )
    else:
        return (T * pow(P0/p, chi) * pow(1.-(rl+ri)/(EPS+rt), chi)
                * exp((-LV*rl-LS*ri) / (CPD+rt*CPV) / T) )

# theta_l_calc (liquid water eq. potential temperature) in Fiaz's code.
@njit
def eval_theta_l(T, q, ql, p,
                 RD=rd,
                 RV=rv,
                 CPD=cpd,
                 CPV=cpv,
                 EPS=epsilon,
                 P0=1e+05):
    """Calculate liquid water potential temperature.
    
    Units: K.
    
    Args:
        T (double): air temperature in K.
        q (double): specific humidity of water vapor in kg/kg.
        ql (double): specific liquid water content in kg/kg.
        p (double): air pressure in Pa.
        RD (constant): gas constant of dry air in J/Kg/K.
        RV (constant): gas constant of water vapor in J/Kg/K.
        CPD (constant): heat capacity at constant pressure for dry air in J/Kg/K.
        CPV (constant): heat capacity at constant pressure for water vapor in J/Kg/K.
        EPS (constant): ratio of gas constant of dry air to gas constant of water vapor.
        P0 (constant): reference pressure (1,000 hPa) in Pa.
        
    Returns:
        double: liquid water potential temperature following Eq. (4.5.15) in Emanuel (1994)
    
    """
    LV = lv(T)
    
    r = q / (1.-q-ql)
    rl = ql / (1.-q-ql)
    rt = r+rl
    
    chi = (RD + rt*RV) / (CPD + rt*CPV)
    gamma = rt*RV / (CPD + rt*CPV)
    
    if rt>0.:
        return (T * pow(P0/p, chi) * pow(1.-rl/(EPS+rt), chi)
                * pow(1.-rl/rt, -gamma)
                * exp(-LV*rl / (CPD+rt*CPV) / T) )
    else:
        return (T * pow(P0/p, chi) * pow(1.-rl/(EPS+rt), chi)
                * exp(-LV*rl / (CPD+rt*CPV) / T) )

@njit
def eval_theta_v(T, q, ql, p,
                 RD=rd,
                 CPD=cpd,
                 P0=1e+05):
    """Calculate virtual potential temperature.
    
    Units: K.
    
    Args:
        T (double): air temperature in K.
        q (double): specific humidity of water vapor in kg/kg.
        ql (double): specific liquid water content in kg/kg.
        p (double): air pressure in Pa.
        RD (constant): gas constant of dry air in J/Kg/K.
        CPD (constant): heat capacity at constant pressure for dry air in J/Kg/K.
        P0 (constant): reference pressure (1,000 hPa) in Pa.
        
    Returns:
        double: virtual potential temperature following Eq. (4.3.2) in Emanuel (1994).
    
    """
    return eval_Tv(T, q, ql) * pow(P0/p, RD/CPD)

@njit
def eval_qsat(T, p):
    """Calculate saturation specific humidity.
    
    Depending on temperature, this subroutine 
    calls eval_qs (T>273.15K) & eval_qsi (T<=273.15K).
    
    Units: kg/kg (unitless).
    
    Args:
        T (double): air temperature in K.
        p (double): air pressure in Pa.
        EPS (constant): ratio of gas constant of dry air to gas constant of water vapor.
        
    Returns:
        double: saturation specific humidity.
    """
    if T>273.15:
        return eval_qs(T, p)
    else:
        return eval_qsi(T, p)

@njit
def eval_qsi(T, p, EPS=epsilon):
    """Calculate ice saturation specific humidity.
    
    Units: kg/kg (unitless).
    
    Args:
        T (double): air temperature in K.
        p (double): air pressure in Pa.
        EPS (constant): ratio of gas constant of dry air to gas constant of water vapor.
        
    Returns:
        double: ice saturation specific humidity.
    """
    e = eval_esi(T)
    
    return EPS * e / ( p-e*(1.-EPS) )

@njit
def eval_qs(T, p, EPS=epsilon):
    """Calculate saturation specific humidity with respect to liquid water.
    
    Units: kg/kg (unitless).
    
    Args:
        T (double): air temperature in K.
        p (double): air pressure in Pa.
        EPS (constant): ratio of gas constant of dry air to gas constant of water vapor.
        
    Returns:
        double: saturation specific humidity.
    """
    e = eval_es(T)
    
    return EPS * e / ( p-e*(1.-EPS) )

@njit
def eval_q(e, p, EPS=epsilon):
    """Calculate specific humidity.
    
    Units: kg/kg (unitless).
    
    Args:
        e (double): vapor pressure in Pa.
        p (double): air pressure in Pa.
        EPS (constant): ratio of gas constant of dry air to gas constant of water vapor.
        
    Returns:
        double: specific humidity.
    """   
    return EPS * e / ( p-e*(1.-EPS) )

@njit
def eval_esi(T):
    """Calculate ice saturation vapor pressure.
    
    Units: Pa.
    
    Accuracy within 0.14% for -80<=T<=0-deg-C.
    
    Args:
        T (double): air temperature in K.
        
    Returns:
        double: ice saturation vapor pressure following Eq. (4.4.15) in Emanuel (1994).
    """    
    return 100. * exp( 23.33086 - 6111.72784/T + 0.15215*log(T) )

@njit
def eval_es(T):
    """Calculate saturation vapor pressure with respect to liquid water.
    
    Units: Pa.
    
    Using 2 formulae for different temperature ranges (separated by -80-deg-C).
    For T>-80-deg-C, the difference between the 2 formulae is within 0.5%.
    
    Args:
        T (double): air temperature in K.
        
    Returns:
        double: saturation vapor pressure of water (liquid).
    """
    
    Tc = T-273.15
    
    if Tc < -80.: # Is < a typo?
        return eval_es_bolton(T)
    else:
        return 0.6105851e+03 + Tc *( 
               0.4440316e+02 + Tc *(
               0.1430341e+01 + Tc *( 
               0.2641412e-01 + Tc *( 
               0.2995057e-03 + Tc *( 
               0.2031998e-05 + Tc *(
               0.6936113e-08 + Tc *(
               0.2564861e-11 - Tc * 0.3704404e-13)))))))

@njit
def eval_es_bolton(T):
    """Calculated saturation vapor pressure with respect to liquid water following Bolton (1980).
    
    Units: Pa.
    
    Accuracy within 0.3% for -35<T<35-deg-C.
    
    Args:
        T (double): air temperature in K.
        
    Returns:
        double: saturation vapor pressure of water (liquid).    
    """
    Tc = T-273.15
    
    return 611.2 * exp( 17.67 * Tc / (243.5+Tc) )

@njit
def eval_rho(T, q, ql, p, RD=rd):
    """Calculate air density.
    
    Units: Kg/m^3.
    
    Virtual (density) temperature is used in the calculation.
    
    Args:
        T (double): air temperature in K.
        q (double): specific humidity of water vapor in kg/kg.
        ql (double): specific liquid water content in kg/kg.
        p (double): air pressure in Pa.
        RD (constant): gas constant of dry air in J/Kg/K.
        
    Returns:
        double: density following Eq. (4.2.8) in Emanuel (1994).
    """
    return p / RD / eval_Tv(T, q, ql)

@njit
def eval_Tv(T, q, ql, EPS=epsilon):
    """Calculate virtual temperature (actually, density temperature).
    
    Units: K.
    
    Vapor and liquid phase of water are included in the calculation.
    
    Args:
        T (double): air temperature in K.
        q (double): specific humidity of water vapor in kg/kg.
        ql (double): specific liquid water content in kg/kg.
        EPS (constant): ratio of gas constant of dry air to gas constant of water vapor.
    
    Returns:
        double: density temperature following Eq. (4.3.6) in Emanuel (1994).
    """
    r = q / (1.-q-ql)
    rl = ql / (1.-q-ql)
    rt = r+rl
    
    return T * (1.+r/EPS) / (1.+rt)

def conv_w2omega(w, T, q, ql, p, G=g):
    """Convert vertical velocity w to omega.
    
    Units: Pa/s.
    
    Assuming hydrostatic balance.
    
    Args:
        w (double): vertical velocity in m/s.
        T (double): air temperature in K.
        q (double): specific humidity of water vapor in kg/kg.
        ql (double): specific liquid water content in kg/kg.
        p (double): air pressure in Pa.
        G (constant): standard gravity.
        
    Returns:
        double: vertical velocity omega = -rho*G*w in Pa/s.
    """
    return -G * eval_rho(T, q, ql, p) * w
#==============================calculate other index(CTP/HI, HCF)
def cal_ctp_hi(temperature,dewpoint,pressure,altitude):
	'''
	INPUT:
		temperature 
		dewpoint
		pressure
		altitude 
  Note: all input should have metpy.units but altitude
	OUTPUT:
		ctp  : Convectie trigering Potential
		hi	  : Humidity index
		moist_t : moist adiabatic temperature profile from 1km to 3km AGL
		cal_p   : Pressure profile from 1km to 3km AGL
	'''
	alt = altitude    
	start_index = np.where(alt == alt[0]+1000)[0][0]
	end_index   = np.where(alt == alt[0]+3000)[0][0]
	cal_p = pressure[start_index:end_index+1]
	cal_t = temperature[start_index:end_index+1]

	moist_t = moist_lapse(pressure = cal_p, 
	            temperature = cal_t[0])
	p_level = np.arange(1000,500,-2)*units('hPa')
	moist_temp = moist_lapse(pressure = p_level, 
	            temperature = cal_t[0],
	            ref_pressure = cal_p[0]).to(units.degC)
	moist_t = np.interp(cal_p.magnitude[::-1],p_level.magnitude[::-1],moist_temp.magnitude[::-1])[::-1]*moist_temp.units
	#ctp0 = np.trapz((moist_t-cal_t)[::-1],cal_p[::-1])
	ctp1 = (mpconsts.Rd*(np.trapz((moist_t.magnitude-cal_t.magnitude)[::-1],np.log(cal_p.magnitude)[::-1])* units.degK)).to(units('J/kg'))    

	index0 = np.where(alt == alt[0]+500)[0][0]
	index1 = np.where(alt == alt[0]+1500)[0][0]
	hi  = temperature[index0]-dewpoint[index0]+temperature[index1]-dewpoint[index1]
	return ctp1.magnitude,hi.magnitude,moist_t.magnitude,cal_p
#=====================================================
def CAL_total_CTP_HI(temperature,dewpoint,pressure,altitude):
	# cal CTP/HI for matrix
	i = 0
	CTP,HI,Moist_t,Cal_p = cal_ctp_hi(temperature[i],dewpoint[i],pressure[i],altitude)
	for i in range(1,len(temperature)):
		CTP_,HI_,Moist_t_,Cal_p_ = cal_ctp_hi(temperature[i],dewpoint[i],pressure[i],altitude)
		CTP = np.append(CTP,CTP_)
		HI  = np.append(HI ,HI_)
		Moist_t = np.concatenate((Moist_t,Moist_t_),axis = 0)
		Cal_p   = np.concatenate((Cal_p,Cal_p_),axis = 0)
	return CTP,HI,Moist_t, Cal_p
#=============================================================
def plot_bg(xlabel,ylabel,xlim = None,ylim = None,grid = 1,label_size = 20,tick_size = 15,ax = None):
	if ax == None:
		ax = plt.gca()
	wide_spines(ax)
	if grid:
		plt.grid(linestyle = ':',color = 'k')
	plt.xlabel(xlabel,fontsize = label_size)
	plt.ylabel(ylabel,fontsize = label_size)
	plt.xticks(fontsize= tick_size)
	plt.yticks(fontsize= tick_size)
	if not xlim == None:
		plt.xlim(xlim)
	if not ylim == None:
		plt.ylim(ylim)

def plot_ctp_hi(ctp,hi,color = 'k'):
    plt.scatter(ctp,hi,c = color)
    
    plt.axvline(x = 0,color = 'k',linewidth = 2)
    plt.scatter(ctp,hi,
                marker = '8',
                c = color,
                #c = integ_index0,cmap = 'RdYlBu',
                #vmax = 100,vmin = -100,
                edgecolors='k')
    
    pick = (ctp<0)
    plt.annotate('Too stable to rain',xy = (0.01,0.1),xycoords = 'axes fraction',fontsize = 15)
    plt.annotate('(%.2f%%)'%(100*sum(pick)/len(ctp)),xy = (0.01,0.05),xycoords = 'axes fraction',fontsize = 15)

    pick = (ctp>=0)&(hi>15)
    plt.annotate('Too dry to rain',xy = (0.75,0.95 ),xycoords = 'axes fraction',fontsize = 15)
    plt.annotate('(%.2f%%)'%(100*sum(pick)/len(ctp)),xy = (0.75,0.9),xycoords = 'axes fraction',fontsize = 15)

    pick = (ctp>=0)&(hi>10)&(hi<=15)
    plt.annotate('DA',xy = (0.75,0.23),xycoords = 'axes fraction',fontsize = 15)
    plt.annotate('(%.2f%%)'%(100*sum(pick)/len(ctp)),xy = (0.85,0.2),xycoords = 'axes fraction',fontsize = 15)

    pick = (ctp>=0)&(hi<=10)&(hi>5)
    plt.annotate('WA',xy = (0.75,0.14),xycoords = 'axes fraction',fontsize = 15)
    plt.annotate('(%.2f%%)'%(100*sum(pick)/len(ctp)),xy = (0.85,0.11),xycoords = 'axes fraction',fontsize = 15)

    pick = (ctp>=0)&(hi<=5)
    plt.annotate('Rain',xy = (0.75,0.05),xycoords = 'axes fraction',fontsize = 15) 
    plt.annotate('(%.2f%%)'%(100*sum(pick)/len(ctp)),xy = (0.85,0.01),xycoords = 'axes fraction',fontsize = 15)


    plt.hlines(y=5,xmin = 0,xmax = 700,colors = 'k')
    plt.hlines(y=10,xmin = 0,xmax = 700,colors = 'k')
    plt.hlines(y=15,xmin = 0,xmax = 700,colors = 'k')
    plt.axvline(x=0,c = 'k')
    plt.vlines(x=200,ymin = 0,ymax = 5,colors = 'k')
#     plt.xlim([-1200,500])
#     plt.ylim([0,60])
    plot_bg(xlabel = '$CTP(J/kg)$',ylabel = '$HI_{low}(C)$',xlim = [-500,400],ylim = [0,70])
#     plt.legend(loc = 2,fontsize = 15)
    plt.tight_layout()


def cal_hcf(temperature,dewpoint,pressure,alt):
    '''
    INPUT:
      temperature
      dewpoint
      pressure
      alt
    Note: all variables should with metpy units

    OUTPUT:
      pp      : pressure of BCLf
      hh      : height of BCL
      theta   : potential temperature of BCL
      theta0  : potential temperature of surface
    '''

    sh_ = specific_humidity_from_mixing_ratio(saturation_mixing_ratio(temperature=dewpoint,tot_press=pressure))
    sh_a = np.array([np.nanmean(sh_[:k]) for k in range(len(pressure))])
    sh__ = []
    for i in range(len(temperature)):
        sh__ = np.append(sh__,specific_humidity_from_mixing_ratio(saturation_mixing_ratio(tot_press = pressure[i],
                                temperature = temperature[i]).magnitude))            
    pp,shh = find_intersections(pressure[1:],sh_a[1:],sh__[1:],direction='increasing')
    tt,shh = find_intersections(temperature[1:],sh_a[1:],sh__[1:],direction='increasing')
    hh,shh = find_intersections(alt[1:],sh_a[1:],sh__[1:],direction='increasing')        
    theta = potential_temperature(pressure = pp, temperature = tt)
    theta0 = potential_temperature(pressure = pressure[0], temperature = temperature[0])
    if len(pp)==0:
        return np.nan,np.nan,np.nan,np.nan
    else:
        return pp[0].magnitude,hh[0].magnitude,theta[0].magnitude,theta0.magnitude
## partial correaltion
"""
Partial Correlation in Python (clone of Matlab's partialcorr)
This uses the linear regression approach to compute the partial 
correlation (might be slow for a huge number of variables). The 
algorithm is detailed here:
    http://en.wikipedia.org/wiki/Partial_correlation#Using_linear_regression
Taking X and Y two variables of interest and Z the matrix with all the variable minus {X, Y},
the algorithm can be summarized as
    1) perform a normal linear least-squares regression with X as the target and Z as the predictor
    2) calculate the residuals in Step #1
    3) perform a normal linear least-squares regression with Y as the target and Z as the predictor
    4) calculate the residuals in Step #3
    5) calculate the correlation coefficient between the residuals from Steps #2 and #4; 
    The result is the partial correlation between X and Y while controlling for the effect of Z
Date: Nov 2014
Author: Fabian Pedregosa-Izquierdo, f@bianp.net
Testing: Valentina Borghesani, valentinaborghesani@gmail.com
"""

from scipy import stats, linalg

def partial_corr(C):
    """
    Returns the sample linear partial correlation coefficients between pairs of variables in C, controlling 
    for the remaining variables in C.
    Parameters
    ----------
    C : array-like, shape (n, p)
        Array with the different variables. Each column of C is taken as a variable
    Returns
    -------
    P : array-like, shape (p, p)
        P[i, j] contains the partial correlation of C[:, i] and C[:, j] controlling
        for the remaining variables in C.
    """
    
    C = np.asarray(C)
    p = C.shape[1]
    P_corr = np.zeros((p, p), dtype=np.float)
    for i in range(p):
        P_corr[i, i] = 1
        for j in range(i+1, p):
            idx = np.ones(p, dtype=np.bool)
            idx[i] = False
            idx[j] = False
            beta_i = linalg.lstsq(C[:, idx], C[:, j])[0]
            beta_j = linalg.lstsq(C[:, idx], C[:, i])[0]

            res_j = C[:, j] - C[:, idx].dot( beta_i)
            res_i = C[:, i] - C[:, idx].dot(beta_j)
            
            corr = stats.pearsonr(res_i, res_j)[0]
            P_corr[i, j] = corr
            P_corr[j, i] = corr
        
    return P_corr

def integrate_buoyancy(temperature,parcel_temperature_profile,pressure,alt,alt_limit = [1000,3000]):  
    y = (parcel_temperature_profile - temperature).to(units.degK) 
    x = pressure.magnitude
    start_level = alt_limit[0]
    end_level   = alt_limit[1]
    p_mask = _less_or_close(alt, end_level) & _greater_or_close(alt, start_level)
    x_clipped = x[p_mask]
    y_clipped = y[p_mask]
    integ_b = np.append(integ_b,(mpconsts.Rd*(np.trapz(y_clipped[::-1], np.log(x_clipped)[::-1]) * units.degK)).to(units('J/kg')))
    return integ_b

def cal_PBLH(theta,alt,p):
    pbl_alt = []
    pbl_p   = []
    for i in range(len(theta)):
        theta_m = theta[i,10:]-theta[i,:-10]
        if not len(np.where((theta[i,1:]-theta[i,:-1])>.25))== 0:        
            pbl_alt= np.append(pbl_alt,alt[np.where(theta_m>.25)[0][0]+10])
            pbl_p  = np.append(pbl_p  ,p[i][np.where(theta_m>.25)[0][0]+10])
    return pbl_alt,pbl_p